In [0]:
#!pip install ray[rllib]==0.7.3 # Originally developed with ver 0.7.3
!pip install ray[rllib]
#!pip install ray[debug]==0.7.3
#!pip install ray[debug]

In [0]:
!pip show tensorflow
!pip show ray

Name: tensorflow
Version: 1.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: opt-einsum, tensorflow-estimator, wheel, keras-preprocessing, numpy, six, keras-applications, google-pasta, grpcio, termcolor, absl-py, wrapt, gast, tensorboard, protobuf, astor
Required-by: stable-baselines, magenta, fancyimpute
Name: ray
Version: 0.8.2
Summary: A system for parallel and distributed Python that unifies the ML ecosystem.
Home-page: https://github.com/ray-project/ray
Author: Ray Team
Author-email: ray-dev@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: pytest, filelock, numpy, click, redis, jsonschema, google, pyyaml, six, py-spy, packaging, protobuf, aiohttp, grpcio, funcsigs, colorama, cloudpickle
Required-by: 


In [0]:
# Needed to switch directory in Google drive so as to import MARL env.

from google.colab import drive 
drive.mount('/content/gdrive')

%cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/"
#%cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/gym_continuousDoubleAuction/"

!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction
/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction


In [0]:
#CDA_env_disc_RLlib

# https://github.com/ray-project/ray/blob/master/python/ray/rllib/examples/multiagent_cartpole.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
"""Simple example of setting up a multi-agent policy mapping.
Control the number of agents and policies via --num-agents and --num-policies.
This works with hundreds of agents and policies, but note that initializing
many TF policies will take some time.
Also, TF evals might slow down with large numbers of policies. To debug TF
execution, set the TF_TIMELINE_DIR environment variable.
"""

# rllib rollout ~/ray_results/PPO/PPO_MMenv-v0_0_2019-08-09_00-06-10t4b1lscr/checkpoint-1 --run PPO --env MMenv-v0 --steps 100

import os
os.environ['RAY_DEBUG_DISABLE_MEMORY_MONITOR'] = "True"

import argparse
import gym
import random
import numpy as np

import ray
from ray import tune
from ray.rllib.models import Model, ModelCatalog

from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet_v2 import FullyConnectedNetwork

from ray.tune.registry import register_env
from ray.rllib.utils import try_import_tf


from ray.rllib.policy.policy import Policy
#from ray.rllib.policy.tf_policy import TFPolicy
#from ray.rllib.policy import Policy


#from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG


import sys
if "../" not in sys.path:
    sys.path.append("../")
#from exchg.x.y import z

#from envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv
from gym_continuousDoubleAuction.envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv

tf = try_import_tf()

In [0]:
parser = argparse.ArgumentParser()
parser.add_argument("--num-agents", type=int, default=4)
parser.add_argument("--num-policies", type=int, default=4)
parser.add_argument("--num-iters", type=int, default=10)
parser.add_argument("--simple", action="store_true")

_StoreTrueAction(option_strings=['--simple'], dest='simple', nargs=0, const=True, default=False, type=None, choices=None, help=None, metavar=None)

In [0]:
class CustomModel_disc(Model):
    def _lstm(self, Inputs, cell_size):
        s = tf.expand_dims(Inputs, axis=1, name='time_major')  # [time_step, feature] => [time_step, batch, feature]
        lstm_cell = tf.nn.rnn_cell.LSTMCell(cell_size)
        self.init_state = lstm_cell.zero_state(batch_size=1, dtype=tf.float32)
        # time_major means [time_step, batch, feature] while batch major means [batch, time_step, feature]
        outputs, self.final_state = tf.nn.dynamic_rnn(cell=lstm_cell, inputs=s, initial_state=self.init_state, time_major=True)
        lstm_out = tf.reshape(outputs, [-1, cell_size], name='flatten_rnn_outputs')  # joined state representation
        return lstm_out

    def _build_layers_v2(self, input_dict, num_outputs, options):
        hidden = 512
        cell_size = 256
        #S = input_dict["obs"]
        S = tf.layers.flatten(input_dict["obs"])
        with tf.variable_scope(tf.VariableScope(tf.AUTO_REUSE, "shared"),
                               reuse=tf.AUTO_REUSE,
                               auxiliary_name_scope=False):
            last_layer = tf.layers.dense(S, hidden, activation=tf.nn.relu, name="fc1")
        last_layer = tf.layers.dense(last_layer, hidden, activation=tf.nn.relu, name="fc2")
        last_layer = tf.layers.dense(last_layer, hidden, activation=tf.nn.relu, name="fc3")

        last_layer = self._lstm(last_layer, cell_size)

        output = tf.layers.dense(last_layer, num_outputs, activation=tf.nn.softmax, name="mu")

        return output, last_layer

In [0]:
class CustomModel(TFModelV2):
    """Example of a custom model that just delegates to a fc-net."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        
        super(CustomModel, self).__init__(obs_space, action_space, num_outputs, model_config, name)
        
        self.model = FullyConnectedNetwork(obs_space, action_space, num_outputs, model_config, name)
        #print('obs_space', obs_space)

        self.register_variables(self.model.variables())

    def forward(self, input_dict, state, seq_lens):
        return self.model.forward(input_dict, state, seq_lens)

    def value_function(self):
        return self.model.value_function()

In [0]:
class MyKerasModel(TFModelV2):
    """Custom model for policy gradient algorithms."""

    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        super(MyKerasModel, self).__init__(obs_space, action_space,
                                           num_outputs, model_config, name)
        self.inputs = tf.keras.layers.Input(
            shape=obs_space.shape, name="observations")
        layer_1 = tf.keras.layers.Dense(
            128,
            name="my_layer1",
            activation=tf.nn.relu,
            kernel_initializer=normc_initializer(1.0))(self.inputs)
        layer_out = tf.keras.layers.Dense(
            num_outputs,
            name="my_out",
            activation=None,
            kernel_initializer=normc_initializer(0.01))(layer_1)
        value_out = tf.keras.layers.Dense(
            1,
            name="value_out",
            activation=None,
            kernel_initializer=normc_initializer(0.01))(layer_1)
        self.base_model = tf.keras.Model(self.inputs, [layer_out, value_out])
        self.register_variables(self.base_model.variables)

    def forward(self, input_dict, state, seq_lens):
        model_out, self._value_out = self.base_model(input_dict["obs"])
        return model_out, state

    def value_function(self):
        return tf.reshape(self._value_out, [-1])

In [0]:
class FastModel(Model):
    def _build_layers_v2(self, input_dict, num_outputs, options):
        bias = tf.get_variable(
            dtype=tf.float32,
            name="bias",
            initializer=tf.zeros_initializer,
            shape=())
        output = bias + tf.zeros([tf.shape(input_dict["obs"])[0], num_outputs])
        return output, output

In [0]:
def make_RandomPolicy(_seed):

    # a hand-coded policy that acts at random in the env (doesn't learn)
    class RandomPolicy(Policy):
        """Hand-coded policy that returns random actions."""
        def __init__(self, observation_space, action_space, config):
            self.observation_space = observation_space
            self.action_space = action_space
            self.action_space.seed(_seed)

        def compute_actions(self,
                            obs_batch,
                            state_batches,
                            prev_action_batch=None,
                            prev_reward_batch=None,
                            info_batch=None,
                            episodes=None,
                            **kwargs):
            """Compute actions on a batch of observations."""
            return [self.action_space.sample() for _ in obs_batch], [], {}

        def learn_on_batch(self, samples):
            """No learning."""
            #return {}
            pass

        def get_weights(self):
            pass

        def set_weights(self, weights):
            pass

    return RandomPolicy

In [0]:
num_agents = 4
num_policies = 4
num_iters = 3
simple = False#store_true

In [0]:
#args = parser.parse_args()

#ray.init(ignore_reinit_error=True)
#ray.init(ignore_reinit_error=True, num_cpus=2)
#ray.init(ignore_reinit_error=True, webui_host='127.0.0.1', num_cpus=2)
ray.init(ignore_reinit_error=True, log_to_driver=False, webui_host='127.0.0.1', num_cpus=2)

#num_of_traders = args.num_agents
num_of_traders = num_agents
tape_display_length = 100
tick_size = 1
init_cash = 1000000
max_step = 700 # per episode
episode = 9
CDA_env = continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step)
print('CDA_env:', CDA_env.print_accs())
register_env("continuousDoubleAuction-v0", lambda _: continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step))

ModelCatalog.register_custom_model("model_disc", CustomModel_disc)
#ModelCatalog.register_custom_model("model_disc", CustomModel)
#ModelCatalog.register_custom_model("model_disc", MyKerasModel)
#ModelCatalog.register_custom_model("model_disc", FastModel)

obs_space = CDA_env.observation_space
act_space = CDA_env.action_space

2020-02-29 19:23:54,331	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-02-29 19:23:54,334	INFO resource_spec.py:212 -- Starting Ray with 6.74 GiB memory available for workers and up to 3.37 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-02-29 19:23:54,981	INFO services.py:1078 -- View the Ray dashboard at 127.0.0.1:8265


ID: 0
cash: 1000000
cash_on_hold: 0
position_val: 0
prev_nav: 1000000
nav: 1000000
net_position: 0
VWAP: 0
profit from trade(tick): 0
total_profit: 0
num_trades: 0
ID: 1
cash: 1000000
cash_on_hold: 0
position_val: 0
prev_nav: 1000000
nav: 1000000
net_position: 0
VWAP: 0
profit from trade(tick): 0
total_profit: 0
num_trades: 0
ID: 2
cash: 1000000
cash_on_hold: 0
position_val: 0
prev_nav: 1000000
nav: 1000000
net_position: 0
VWAP: 0
profit from trade(tick): 0
total_profit: 0
num_trades: 0
ID: 3
cash: 1000000
cash_on_hold: 0
position_val: 0
prev_nav: 1000000
nav: 1000000
net_position: 0
VWAP: 0
profit from trade(tick): 0
total_profit: 0
num_trades: 0
CDA_env: 0


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
# Each policy can have a different configuration (including custom model)
def gen_policy(i):
    config = {"model": {"custom_model": "model_disc"},
              "gamma": 0.99,}
    return (None, obs_space, act_space, config)

In [0]:
def policy_mapper(agent_id):
    if agent_id == 0:
        return "policy_0" # PPO
    elif agent_id == 1:
        return "policy_1" # RandomPolicy
    elif agent_id == 2:
        return "policy_2" # RandomPolicy
    else:
        return "policy_3" # RandomPolicy

In [0]:
# Setup PPO with an ensemble of `num_policies` different policies

# Dictionary of policies
#policies = {"policy_{}".format(i): gen_policy(i) for i in range(args.num_policies)}
policies = {"policy_{}".format(i): gen_policy(i) for i in range(num_policies)}

# override policy with random policy
#policies["policy_{}".format(args.num_policies-3)] = (make_RandomPolicy(1), obs_space, act_space, {}) # random policy stored as the last item in policies dictionary
#policies["policy_{}".format(args.num_policies-2)] = (make_RandomPolicy(2), obs_space, act_space, {}) # random policy stored as the last item in policies dictionary
#policies["policy_{}".format(args.num_policies-1)] = (make_RandomPolicy(3), obs_space, act_space, {}) # random policy stored as the last item in policies dictionary
policies["policy_{}".format(num_policies-3)] = (make_RandomPolicy(1), obs_space, act_space, {}) # random policy stored as the last item in policies dictionary
policies["policy_{}".format(num_policies-2)] = (make_RandomPolicy(2), obs_space, act_space, {}) # random policy stored as the last item in policies dictionary
policies["policy_{}".format(num_policies-1)] = (make_RandomPolicy(3), obs_space, act_space, {}) # random policy stored as the last item in policies dictionary

print('policies:', policies)

policy_ids = list(policies.keys())

policies: {'policy_0': (None, Box(4, 10), Tuple(Discrete(3), Discrete(4), Discrete(100), Discrete(12)), {'model': {'custom_model': 'model_disc'}, 'gamma': 0.99}), 'policy_1': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Discrete(100), Discrete(12)), {}), 'policy_2': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Discrete(100), Discrete(12)), {}), 'policy_3': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Discrete(100), Discrete(12)), {})}


In [0]:
tune.run(#PPOTrainer,
          "PPO",
          #"PG",
          #queue_trials=False,
          #resources_per_trial={"cpu": 2,
          #                     "gpu": 0},
         
          #stop={"training_iteration": args.num_iters},
          #stop={"timesteps_total": max_step,
          #      "training_iteration": num_iters},
          stop={"timesteps_total": max_step * episode},
         
          config={"env": "continuousDoubleAuction-v0",
                  
                  #"log_level": "DEBUG",
                  #"simple_optimizer": args.simple,
                  #"simple_optimizer": True,
                  #"num_sgd_iter": 10,
                  
                  #"gamma": 0.9,

                  # Number of rollout worker actors to create for parallel sampling.                   
                  # Setting to 0 will force rollouts to be done in the trainer actor.
                  "num_workers": 1, # Colab (only 2 CPUs or 1 GPU)                  
                  "num_envs_per_worker": 2, #4

                  #"timesteps_per_iteration": max_step,

                  # https://github.com/ray-project/ray/issues/4628
                  "sample_batch_size": 32, # number of environment steps sampled from each environment
                  "train_batch_size": 128, # minibatch size must be >= 128, number of environment steps sampled from all available environments
                                    
                  "multiagent": {"policies_to_train": ["policy_0"],
                                 "policies": policies,
                                 #"policy_mapping_fn": tune.function(lambda agent_id: random.choice(policy_ids)),
                                 #"policy_mapping_fn": tune.function(policy_mapper),
                                 "policy_mapping_fn": policy_mapper,
                                },                                 
                  },
          )

#ray.shutdown()

Trial name,status,loc
PPO_continuousDoubleAuction-v0_07001334,RUNNING,


Result for PPO_continuousDoubleAuction-v0_07001334:
  custom_metrics: {}
  date: 2020-02-29_19-24-34
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: 0a648881c4bc4a878ccbbb1728f6c82e
  experiment_tag: '0'
  hostname: 7e6fe8db4693
  info:
    grad_time_ms: 15971.467
    learner:
      policy_0:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 9.564391136169434
        entropy_coeff: 0.0
        kl: 0.010105578228831291
        policy_loss: -0.05712352693080902
        total_loss: 389637.3125
        vf_explained_var: 0.0
        vf_loss: 389637.34375
    load_time_ms: 148.68
    num_steps_sampled: 128
    num_steps_trained: 128
    sample_time_ms: 884.733
    update_time_ms: 2026.962
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_pe

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_07001334,RUNNING,172.28.0.2:12191,nan,19.1895,128,1


Result for PPO_continuousDoubleAuction-v0_07001334:
  custom_metrics: {}
  date: 2020-02-29_19-24-50
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: 0a648881c4bc4a878ccbbb1728f6c82e
  experiment_tag: '0'
  hostname: 7e6fe8db4693
  info:
    grad_time_ms: 15407.081
    learner:
      policy_0:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 9.554363250732422
        entropy_coeff: 0.0
        kl: 0.01714286580681801
        policy_loss: -0.05367262661457062
        total_loss: 4390097.5
        vf_explained_var: 0.0
        vf_loss: 4390097.0
    load_time_ms: 74.942
    num_steps_sampled: 256
    num_steps_trained: 256
    sample_time_ms: 803.601
    update_time_ms: 1029.599
  iterations_since_restore: 2
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent:

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_07001334,RUNNING,172.28.0.2:12191,nan,34.7918,256,2


...

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_07001334,RUNNING,172.28.0.2:12191,0,775.435,6272,49


Result for PPO_continuousDoubleAuction-v0_07001334:
  custom_metrics: {}
  date: 2020-02-29_19-37-29
  done: true
  episode_len_mean: 701.0
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 0
  episodes_total: 8
  experiment_id: 0a648881c4bc4a878ccbbb1728f6c82e
  experiment_tag: '0'
  hostname: 7e6fe8db4693
  info:
    grad_time_ms: 14882.699
    learner:
      policy_0:
        cur_kl_coeff: 0.7593749761581421
        cur_lr: 4.999999873689376e-05
        entropy: 9.55988883972168
        entropy_coeff: 0.0
        kl: 0.007329336833208799
        policy_loss: -0.07236185669898987
        total_loss: 21397754.0
        vf_explained_var: 0.0
        vf_loss: 21397754.0
    load_time_ms: 1.38
    num_steps_sampled: 6400
    num_steps_trained: 6400
    sample_time_ms: 740.817
    update_time_ms: 32.836
  iterations_since_restore: 50
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 96

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_07001334,TERMINATED,,0,791.032,6400,50


Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_07001334,TERMINATED,,0,791.032,6400,50


2020-02-29 19:37:29,221	INFO tune.py:352 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.


In [0]:
'''
next_states:
{
    0: [array([ 16., 114., 141.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]), 
        array([4., 3., 2., 0., 0., 0., 0., 0., 0., 0.]), 
        array([  -7.,  -60., -135.,  -97.,  -61., -151., -136.,    0.,    0., 0.]), 
        array([  -7.,   -9.,  -16., -100., -101., -102., -103.,    0.,    0., 0.])], 
 
    1: [array([ 16., 114., 141.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]), 
        array([4., 3., 2., 0., 0., 0., 0., 0., 0., 0.]), 
        array([  -7.,  -60., -135.,  -97.,  -61., -151., -136.,    0.,    0., 0.]), 
        array([  -7.,   -9.,  -16., -100., -101., -102., -103.,    0.,    0., 0.])], 
 
    2: [array([ 16., 114., 141.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]), 
        array([4., 3., 2., 0., 0., 0., 0., 0., 0., 0.]), 
        array([  -7.,  -60., -135.,  -97.,  -61., -151., -136.,    0.,    0., 0.]), 
        array([  -7.,   -9.,  -16., -100., -101., -102., -103.,    0.,    0., 0.])], 
 
    3: [array([ 16., 114., 141.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]), 
        array([4., 3., 2., 0., 0., 0., 0., 0., 0., 0.]), 
        array([  -7.,  -60., -135.,  -97.,  -61., -151., -136.,    0.,    0., 0.]), 
        array([  -7.,   -9.,  -16., -100., -101., -102., -103.,    0.,    0., 0.])]
 }
 '''

'\nnext_states:\n{\n    0: [array([ 16., 114., 141.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]), \n        array([4., 3., 2., 0., 0., 0., 0., 0., 0., 0.]), \n        array([  -7.,  -60., -135.,  -97.,  -61., -151., -136.,    0.,    0., 0.]), \n        array([  -7.,   -9.,  -16., -100., -101., -102., -103.,    0.,    0., 0.])], \n \n    1: [array([ 16., 114., 141.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]), \n        array([4., 3., 2., 0., 0., 0., 0., 0., 0., 0.]), \n        array([  -7.,  -60., -135.,  -97.,  -61., -151., -136.,    0.,    0., 0.]), \n        array([  -7.,   -9.,  -16., -100., -101., -102., -103.,    0.,    0., 0.])], \n \n    2: [array([ 16., 114., 141.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]), \n        array([4., 3., 2., 0., 0., 0., 0., 0., 0., 0.]), \n        array([  -7.,  -60., -135.,  -97.,  -61., -151., -136.,    0.,    0., 0.]), \n        array([  -7.,   -9.,  -16., -100., -101., -102., -103.,    0.,    0., 0.])], \n \n    3: [array([ 16., 114., 141.,   0.,